In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
from PIL import Image
import os
import cv2
from keras import preprocessing
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from skimage import exposure

In [2]:
trainingDataHEM = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_1/hem'
trainingDataALL = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_1/all'


validationDataHEM = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_0/hem'
validationDataALL = 'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/training_data/fold_0/all'

In [3]:
def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.bmp')]

In [4]:
def crop_img(images, display=False):
    mask = images == 0

    coords = np.array(np.nonzero(~mask))
    top_left = np.min(coords, axis=1)
    bottom_right = np.max(coords, axis=1)

    cropped_image = images[top_left[0]:bottom_right[0],top_left[1]:bottom_right[1]]
    return cropped_image

In [5]:
def dataImport(path,pathType):
    imageList = []
    for i,j in zip(pathType,path):
        if i not in ['ALL','HEM']:
            raise ValueError("Entry must be either 'ALL' or 'HEM'")
        else:
            if i == 'HEM':
                loadPath = get_imlist(j) 
                for paths in loadPath:
                    imageList.append({'Images':Image.open(paths),'Labels':0,'CroppedImages':None})
            else:
                loadPath = get_imlist(j) 
                for paths in loadPath:
                    imageList.append({'Images':Image.open(paths),'Labels':1,'CroppedImages':None})
    imageData = pd.DataFrame(imageList)
    imageData['CroppedImages'] = imageData['Images'].apply(lambda images: crop_img(np.array(images)))
    return imageData

In [6]:
trainingData = dataImport([trainingDataALL,trainingDataHEM],['ALL','HEM'])
validationData = dataImport([validationDataALL,validationDataHEM],['ALL','HEM'])

In [7]:
print(trainingData.shape)
print(validationData.shape)

(3581, 3)
(3527, 3)


In [8]:
oneData = pd.concat([trainingData,validationData],axis=0,ignore_index=True)

In [9]:
oneDataALL = oneData[oneData['Labels']==1].reset_index(drop=True)
oneDataHEM = oneData[oneData['Labels']==0].reset_index(drop=True)

In [10]:
oneDataALL = list(oneDataALL['CroppedImages'])
oneDataHEM = list(oneDataHEM['CroppedImages'])

In [11]:
#save croppedImages to drive

In [19]:
os.mkdir('C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/trainingDataHEM')

In [20]:
for index, img in enumerate(oneDataHEM):
    image = img.astype(np.uint8)
    cv2.imwrite(f'C:/Users/v4run/OneDrive/Desktop/Datasets/C-NMC_Leukemia/trainingDataHEM/HEM_{index}.bmp',image)
